In [ ]:
!pip install timm

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import time
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import os
from PIL import Image
from timm import create_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
cbisddsm = '/content/drive/MyDrive/cbis-ddsm/'
cmmd = '/content/drive/MyDrive/output/'
modelsdir = '/content/drive/MyDrive/combined/'
imagesEnsemble = '/content/drive/MyDrive/combined/imagesEnsemble'

deit = '/content/drive/MyDrive/combined/Deit.pth'
densenet201 = '/content/drive/MyDrive/combined/densenet201.pth'
inceptionv3full = '/content/drive/MyDrive/combined/inceptionv3m_fulltrained.pth'
resnet50 = '/content/drive/MyDrive/combined/resnet50.pth'
vgg16bn = '/content/drive/MyDrive/combined/vgg16bn.pth'
densenet212 = '/content/drive/MyDrive/combined/densenet121.pth'
inceptionv3 = '/content/drive/MyDrive/combined/inceptionv3.pth'
efficientnetB0 = '/content/drive/MyDrive/combined/efficientnetB0.pth'
EnhancedCNN = '/content/drive/MyDrive/output/models/Combined_EnhancedCNN.pth'

In [ ]:
X_train = torch.load(datasetdir + "/X_train_balanced.pt")
y_train = torch.load(datasetdir + "/y_train_balanced.pt")

X_test = torch.load(datasetdir + "/X_test.pt")
y_test = torch.load(datasetdir + "/y_test.pt")

In [ ]:
print('X train shape : {}' .format(X_train.shape))
print('y train shape : {}' .format(y_train.shape))
print('X test shape : {}' .format(X_test.shape))
print('y test shape : {}' .format(y_test.shape))

X train shape : torch.Size([1318, 3, 224, 224])
y train shape : torch.Size([1318])
X test shape : torch.Size([378, 3, 224, 224])
y test shape : torch.Size([378])


In [ ]:
# Necesario para que el modelo entrenado pueda ser cargado, si no, da fallo
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()

        # Primera capa
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),  # Normalización por lotes
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)  # Dropout para reducir el sobreajuste
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.3)
        )

        # Clasificador final
        self.classifier = nn.Sequential(
            nn.Linear(256 * 14 * 14, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
def load_model(path):
    model.load(path)
    model.eval()
    return model

model_paths = {
    'deit': '/content/drive/MyDrive/combined/Deit.pth',
    'inceptionv3full': '/content/drive/MyDrive/combined/inceptionv3m_fulltrained.pth',
    'resnet50': '/content/drive/MyDrive/combined/resnet50.pth',
    'vgg16bn': '/content/drive/MyDrive/combined/vgg16bn.pth',
    'densenet121': '/content/drive/MyDrive/combined/densenet121.pth',
    'inceptionv3': '/content/drive/MyDrive/combined/inceptionv3.pth',
    'efficientnetB0Manual': '/content/drive/MyDrive/combined/efficientnetB0_manual.pth'
    'EnhancedCNN' : '/content/drive/MyDrive/output/models/Combined_EnhancedCNN.pth',
    'CNN-LCNGCN': '/content/drive/MyDrive/output/models/Combined_CNN_LCN_GCN.pth'
}

model_classes = {
    'resnet50': models.resnet50,
    'densenet121': models.densenet121(pretrained=False),
    'efficientnetB0Manual': EfficientNetB0(num_classes=2),
    'EnhancedCNN': EnhancedCNN(),
    'CNN-LCNGCN': EnhancedCNN(),
    'densenet121Pesos': models.densenet121(weights='DenseNet121_Weights.DEFAULT')

}

num_classes = 2
loaded_models = {name: load_model(path) for name, path in model_paths.items()}

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

def predict(models, image_path, weights=None):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)

    if weights is None:
        weights = {name: 1.0 for name in models.keys()}

    total_weight = sum(weights.values())
    weights = {name: weight / total_weight for name, weight in weights.items()}

    predictions = {}
    for name, model in models.items():
        with torch.no_grad():
            output = model(input_tensor)
            predictions[name] = F.softmax(output, dim=1) * weights[name]

    final_prediction = sum(predictions.values())
    _, predicted_class = torch.max(final_prediction, 1)

    return predicted_class.item()

image_path = '/content/drive/MyDrive/output/D2-0748_1-1.png'
realLabel = 1

weights = {
    'resnet50': 0.73,
    'densenet121': 0.82,
    'densenet121Pesos': 0.78,
    'efficientnetB0Manual': 0.81,
    'EnhancedCNN': 0.83,
    'CNN-LCNGCN': 0.83
}
predicted_class = predict(models, image_path, weights)
print(f'Predicted class: {predicted_class}')


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 14 * 14, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
def predict_image(model, image_path, device):
    filas = 224
    columnas = 224

    preprocess = transforms.Compose([
        transforms.Resize((filas, columnas)),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(0.5, 0.5)
    ])

    image_path = os.path.join(image_path)
    img = Image.open(image_path)
    img_tensor = preprocess(img)
    img_tensor = img_tensor.unsqueeze(0).to(device)

    model_path = '/content/drive/MyDrive/OLD_output/models/CombinedCNN.pth'
    model = torch.load(model_path, map_location=device)
    model.eval()

    with torch.no_grad():
        output = model(img_tensor)

    _, predicted_class = torch.max(output, 1)
    return predicted_class.item()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#image_path = '/content/drive/MyDrive/cbis-ddsm/jpeg/1.3.6.1.4.1.9590.100.1.2.245063149211255120613007755642780114172/1-271.jpg' # Debe dar predicción BENIGNO
image_path = '/content/drive/MyDrive/output/images/D2-0748_1-1.png' # Debe dar predicción MALIGNO
predicted_class = predict_image(CNN().to(device), image_path, device)

print(f'Valor de predicción: {predicted_class}')

if predicted_class == 0:
    print("Predicción: BENIGNO")
else:
    print("Predicción: MALIGNO")

Valor de predicción: 1
Predicción: MALIGNO


In [ ]:
def load_models(model_paths, device):
    models = {}
    for name, path in model_paths.items():
        model = torch.load(path, map_location=device)
        model.eval()
        models[name] = model
    return models

def get_preprocess(num_channels):
    filas = 224
    columnas = 224

    if num_channels == 3:
        return transforms.Compose([
            transforms.Resize((filas, columnas)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
    else:
        return transforms.Compose([
            transforms.Resize((filas, columnas)),
            transforms.Grayscale(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5], std=[0.5])
        ])

def collective_predict(models, image_path, device, weights):
    image_path = os.path.join(image_path)
    img = Image.open(image_path).convert('L')

    total_weight = sum(weights.values())
    weights = {name: weight / total_weight for name, weight in weights.items()}

    final_prediction = torch.zeros(1, 2).to(device)

    for name, model in models.items():
        print(f"Procesando modelo: {name}")  # Depuración
        try:
            first_layer = next(model.parameters())
            num_channels = first_layer.shape[1]
        except StopIteration:
            num_channels = 1

        preprocess = get_preprocess(num_channels)
        img_tensor = preprocess(img)
        img_tensor = img_tensor.unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(img_tensor)

            if isinstance(output, tuple):
                output = output[0]

            print(f"Salida del modelo {name}: {output}")  # Depuración

            if output.dim() == 1:
                print(f"Ajustando salida del modelo {name} de {output.shape} a [1, {output.shape[0]}]")
                output = output.unsqueeze(0)


            final_prediction += F.softmax(output, dim=1) * weights[name]
            print(f"Salida calculada predictiva: {final_prediction}"

    _, predicted_class = torch.max(final_prediction, 1)
    #print(f"Salida calculada predictiva: {predicted_class.item()}")  # Depuración
    return predicted_class.item()

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_paths = {
    'deit': '/content/drive/MyDrive/combined/Deit.pth',
    'inceptionv3full': '/content/drive/MyDrive/combined/inceptionv3m_fulltrained.pth',
    'resnet50': '/content/drive/MyDrive/combined/resnet50.pth',
    'vgg16bn': '/content/drive/MyDrive/combined/vgg16bn.pth',
    'densenet121': '/content/drive/MyDrive/combined/densenet121.pth',
    'inceptionv3': '/content/drive/MyDrive/combined/inceptionv3.pth',
    'efficientnetB0Manual': '/content/drive/MyDrive/combined/efficientnetB0_manual.pth'
    'EnhancedCNN' : '/content/drive/MyDrive/output/models/Combined_EnhancedCNN.pth'
}

weights = { # Pesos según F1-Score para la ponderación de la decisión
    'resnet50': 0.73,
    'vgg16bn': 0.61,
    'densenet121': 0.78,
    'inceptionv3': 0.81,
    'efficientnetB0Manual': 0.81,
    'EnhancedCNN': 0.84
}

models = load_models(model_paths, device)
image_path = '/content/drive/MyDrive/output/images/D2-0748_1-1.png'

predicted_class = collective_predict(models, image_path, device, weights)

if predicted_class == 0:
    print("Predicción: BENIGNO")
else:
    print("Predicción: MALIGNO")


Procesando modelo: densenet201
Salida del modelo densenet201: tensor([[-0.3641,  0.3768]])
Salida calculada predictiva: tensor([[0.1137, 0.2384]])
Procesando modelo: inceptionv3full
Salida del modelo inceptionv3full: tensor([[ 0.4031, -0.3669]])
Salida calculada predictiva: tensor([[0.2003, 0.2786]])
Procesando modelo: vgg16bn
Salida del modelo vgg16bn: tensor([[-0.3353,  0.3012]])
Salida calculada predictiva: tensor([[0.2746, 0.4190]])
Procesando modelo: densenet121
Salida del modelo densenet121: tensor([[-0.2768,  1.2441]])
Salida calculada predictiva: tensor([[0.2854, 0.4682]])
Procesando modelo: inceptionv3
Salida del modelo inceptionv3: tensor([[-0.4347,  0.2254]])
Salida calculada predictiva: tensor([[0.2950, 0.4867]])
Procesando modelo: efficientnetB0
Salida del modelo efficientnetB0: tensor([[-0.3660,  0.3568]])
Salida calculada predictiva: tensor([[0.3663, 0.6337]])
Predicción: MALIGNO
